In [49]:
%load_ext autoreload
%autoreload 

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import os
import re
import pickle
from src.data_loader import extract_latex_dependencies
from src.llm_tools import enrich_file_metadata
from src.llm_tools import extract_jd_features

In [42]:
# check for the cache file containing all the dependencies and load it if it exists
cache_dir = "/Users/Bardia/Coding/LangChain/Code_Rsm_Adj/cache"
# check the validity of the cache directory
if not os.path.exists(cache_dir):
    raise FileNotFoundError(f"The directory was not found: ' {cache_dir} ' ")

cache_file_name = "resume_metadata.pkl"
cache_path = os.path.join(cache_dir, cache_file_name)

if os.path.exists(cache_path): 
    with open(cache_path, 'rb') as f: 
        dep_list = pickle.load(f)
        print("File loaded")
#else proccess the latex main file for the dependencies list
else: 
    rsm_main_path = "/Users/Bardia/Rsm/Resume_Bardia_Azami/resume-general/Bardia-Azami-Resume.tex"
    #check the validity of file path 
    if not os.path.exists(rsm_main_path): 
        raise FileNotFoundError(f"The directory was not found: ' {rsm_main_path} ' ")
    #unproccesed dependencies list
    dep_unprc = extract_latex_dependencies(rsm_main_path)
    #enrich the dependencies list 
    dep_list = enrich_file_metadata(dep_unprc)
    print("dependencies extracted")
    #save the list 
    with open(cache_path, 'wb') as f: 
        pickle.dump(dep_list)
        print("The proccesed dependencies saved")


File loaded


In [51]:
# Load the job description and extract key words 
jd_path = "./Rsm/jd/jd.txt"
if not os.path.exists(jd_path):
    raise FileNotFoundError(f"could not find the job description file:{jd_path} ")

jd_ft = extract_jd_features(jd_path)

Step 1: Mining keywords (High Recall)...
Minerd found 79 potential keywords.
Step 2: Judging keywords (High Precision)...
Judge has finished filtering.


In [56]:
print(f"Extracted keywords = \n Domain Knowledge : {jd_ft.domain_knowledge} \n Technical Stack:{jd_ft.technical_stack}, \n tools:{jd_ft.tools_and_platforms}, \n soft_skills:{jd_ft.soft_skills}, \n years of exp:{jd_ft.years_experience_min}")


Extracted keywords = 
 Domain Knowledge : ['Machine Learning', 'Artificial Intelligence', 'Large Language Models', 'Agentic AI', 'Enterprise Software Integration', 'Data Security', 'HTTP', 'WebRTC'] 
 Technical Stack:['Python', 'Rust', 'TypeScript', 'Go', 'PyTorch', 'JAX', 'LLMs'], 
 tools:['AWS', 'GCP', 'Azure', 'MLOps', 'GitHub', 'RAG', 'DeepSearch'], 
 soft_skills:['Problem Solving', 'Detail-Oriented', 'Drive Projects', 'Communication'], 
 years of exp:0
